In [1]:
!python3 --version

Python 3.8.11


In [2]:
import pandas
import numpy
import pycytominer

from utils import calculate_percent_replicating_MOA

In [3]:
def do_feature_select(plate_df):
    feature_select_features = pycytominer.cyto_utils.infer_cp_features(
        plate_df
    )

    return pycytominer.feature_select(
        profiles=plate_df,
        features=feature_select_features,
        operation=['variance_threshold','correlation_threshold',
        'drop_na_columns','blocklist']
    )

In [4]:
normalized_master = pandas.read_csv('../profiles-pilots/profiles/Stain5_CondC_Standard/BR00120274/BR00120274_normalized_negcon.csv.gz')

In [5]:
word_dict ={}
modules = []
for eachcolumn in normalized_master.columns:
    split = eachcolumn.split('_')
    modules.append(split[1])
    for eachsplit in split:
        if eachsplit not in word_dict.keys():
            word_dict[eachsplit] = 1
        else:
            word_dict[eachsplit] += 1
modules=list(set(modules))
modules.sort()
len(word_dict)

136

In [6]:
at_least_21 =[x for x in word_dict.keys() if word_dict[x]>=21]
print(len(at_least_21))
print(at_least_21)

83
['Cells', 'AreaShape', 'X', 'Y', 'Zernike', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'Cytoplasm', 'Correlation', 'AGP', 'BFHigh', 'BFLow', 'Brightfield', 'DNA', 'ER', 'Mito', 'RNA', 'K', 'Manders', 'Overlap', 'RWC', 'Granularity', '10', '11', '12', '13', '14', '15', '16', 'Intensity', 'IntegratedIntensityEdge', 'IntegratedIntensity', 'LowerQuartileIntensity', 'MADIntensity', 'MassDisplacement', 'MaxIntensityEdge', 'MaxIntensity', 'MeanIntensityEdge', 'MeanIntensity', 'MedianIntensity', 'MinIntensityEdge', 'MinIntensity', 'StdIntensityEdge', 'StdIntensity', 'UpperQuartileIntensity', 'Location', 'CenterMassIntensity', 'Z', 'Neighbors', 'Nuclei', 'RadialDistribution', 'FracAtD', '1of4', '2of4', '3of4', '4of4', 'MeanFrac', 'RadialCV', 'Texture', 'AngularSecondMoment', '00', '256', '01', '02', '03', 'Contrast', 'DifferenceEntropy', 'DifferenceVariance', 'Entropy', 'InfoMeas1', 'InfoMeas2', 'InverseDifferenceMoment', 'SumAverage', 'SumEntropy', 'SumVariance', 'Variance']


In [7]:
single_dropout_df = pandas.DataFrame()
for eachdropout in ['CONTROL']+at_least_21:
    col_list = [x for x in normalized_master.columns if eachdropout not in x]
    dropout_dict = {}
    temp_dropout_dict = {}
    for eachplate in range(70,78):
        try:
            normalized = pandas.read_csv(
                f'../profiles-pilots/profiles/Stain5_CondC_Standard/BR001202{eachplate}/BR001202{eachplate}_normalized_negcon.csv.gz',
                usecols=col_list)
            feature_select = do_feature_select(normalized)
            temp_dropout_dict[f'{eachplate}_featnum']=len(feature_select.columns)
            dropout_dict[eachplate]= calculate_percent_replicating_MOA('','',data_df=feature_select)
        except:
            dropout_dict[eachplate]=0
    results_array = numpy.array(list(dropout_dict.values()))
    for k,v in temp_dropout_dict.items():
        dropout_dict[k]=v
    dropout_dict['Mean']=results_array.mean()
    dropout_dict['Median']=numpy.median(results_array)
    dropout_dict['Standard deviation']=numpy.std(results_array)
    dropout_dict['Dropout']=eachdropout
    dropout_dict['n_columns']=len(col_list)
    single_dropout_df = single_dropout_df.append(dropout_dict,ignore_index=True)
    print(eachdropout,dropout_dict['Mean'])
    single_dropout_df.to_csv('checkpoint_csvs/single_dropout.csv',index=False)



CONTROL 0.5444444444444444
Cells 0.55
AreaShape 0.5527777777777778
X 0.5458333333333333
Y 0.5472222222222222
Zernike 0.55
1 0.5541666666666667
2 0.5555555555555556
3 0.5541666666666667
4 0.5388888888888889
5 0.5472222222222222
6 0.5569444444444445
7 0.5430555555555555
8 0.548611111111111
9 0.5527777777777777
Cytoplasm 0.5375
Correlation 0.5486111111111112
AGP 0.5444444444444444
BFHigh 0.5347222222222222
BFLow 0.5611111111111111
Brightfield 0.5513888888888888
DNA 0.5402777777777777
ER 0.5472222222222222
Mito 0.5347222222222222
RNA 0.5472222222222222
K 0.5472222222222222
Manders 0.5416666666666666
Overlap 0.5472222222222222
RWC 0.5472222222222222
Granularity 0.5458333333333333
10 0.5513888888888889
11 0.5430555555555555
12 0.5513888888888888
13 0.5472222222222222
14 0.5472222222222222
15 0.5444444444444444
16 0.5458333333333333
Intensity 0.5444444444444444
IntegratedIntensityEdge 0.5430555555555555
IntegratedIntensity 0.5430555555555555
LowerQuartileIntensity 0.5416666666666666
MADIntens